In [1]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date

### Primeiro, importar os eventos e os perfils para ver a cara desses dados para ver se está tudo OK.

In [2]:
df_eventos = pd.read_csv('data/01-eventos-consolidados_apenas-free.csv', index_col=0)

In [5]:
df_perfils = pd.read_csv('data/02-perfils-higienizados.csv', index_col=0)

In [6]:
df_eventos.head()

,event,time,distinct_id
0,Busca CNPJ,2020-05-03 00:01:27,bqPeiaqE8GpSRuZL6EXewNzVCBuFTAvJ.b5c5e42839712...
1,Busca CNPJ,2020-05-03 00:05:13,bqPeiaqE8GpSRuZL6EXewNzVCBuFTAvJ.b5c5e42839712...
2,Busca CNPJ,2020-05-03 00:09:34,O4nwEphIRGdwTlJoo71hkaHGZsDDtZ6N.9b2a763248cca...
3,Busca CNPJ,2020-05-03 00:09:35,O4nwEphIRGdwTlJoo71hkaHGZsDDtZ6N.9b2a763248cca...
4,Busca CNPJ,2020-05-03 00:09:37,O4nwEphIRGdwTlJoo71hkaHGZsDDtZ6N.9b2a763248cca...


In [17]:
df_eventos.shape

(934355, 3)

In [7]:
df_perfils.head()

,distinct_id,Browser,city,country_code,created,initial_referring_domain,last_seen,os,region,Source
27,WNcleXIIU5aelXWAfsCOWd8ndhaDQdl4.e815aac4fb5b3...,Chrome 83.0,Curitiba,BR,2020-05-27 13:52:19,$direct,2020-05-27 16:52:21,Windows,Parana,CC CNPJ - Painel de Dados - Telefones
28,172604d426f651-0a9105b4048dfb-1a201708-100200-...,Chrome 73.0,Curitiba,BR,2020-05-29 06:00:20,$direct,2020-06-03 15:33:59,Linux,Parana,LA Criar Conta
29,172630063eaca-0e2be0fe39195-58143718-100200-17...,Chrome 83.0,Recife,BR,2020-05-29 18:34:30,consultacnpj.com,2020-05-29 21:35:13,Windows,Pernambuco,CC CNPJ - Painel de Dados - E-mails
30,HXgZYzItf1dg3U9osEk2wNbSlNuA6gEa.6451e1f34b19e...,Chrome 84.0,Curitiba,BR,2020-06-01 11:51:28,leads2b.com,2020-08-06 16:31:43,Windows,Parana,LA Criar Conta
31,KvlhZUYVslEa4GDBdOM0KcbnQg03v4QV.f2f55b40d6d78...,Chrome 83.0,Sao Jose,BR,2020-06-02 17:05:53,$direct,2020-06-03 09:21:16,Windows,Santa Catarina,LA Criar Conta


In [18]:
df_perfils.shape

(9251, 10)

### Agora vamos criar a Recência e a Duração, que são os mais fáceis.

Primeiro passar as colunas de 'created' e 'last_seen' para o formato de DateTime.

In [65]:
df_perfils['created'] = pd.to_datetime(df_perfils['created'])

In [66]:
df_perfils['last_seen'] = pd.to_datetime(df_perfils['last_seen'])

Para a Recência, vamos contar em dias a distancia entre a última vez que foi visto e o fim da série histórica.

In [67]:
df_perfils['recencia'] = (pd.Timestamp('2020-08-13') - df_perfils['last_seen']).dt.days

Para a Duração, vamos contar em dias a distância entre a data de criação daquele usuário e a última vez que foi visto.

In [68]:
df_perfils['duracao'] = (df_perfils['last_seen'] - df_perfils['created']).dt.days

Agora vamos checar se deu tudo certo:

In [69]:
df_perfils.head()

,distinct_id,Browser,city,country_code,created,initial_referring_domain,last_seen,os,region,Source,recencia,duracao
27,WNcleXIIU5aelXWAfsCOWd8ndhaDQdl4.e815aac4fb5b3...,Chrome 83.0,Curitiba,BR,2020-05-27 13:52:19,$direct,2020-05-27 16:52:21,Windows,Parana,CC CNPJ - Painel de Dados - Telefones,77,0
28,172604d426f651-0a9105b4048dfb-1a201708-100200-...,Chrome 73.0,Curitiba,BR,2020-05-29 06:00:20,$direct,2020-06-03 15:33:59,Linux,Parana,LA Criar Conta,70,5
29,172630063eaca-0e2be0fe39195-58143718-100200-17...,Chrome 83.0,Recife,BR,2020-05-29 18:34:30,consultacnpj.com,2020-05-29 21:35:13,Windows,Pernambuco,CC CNPJ - Painel de Dados - E-mails,75,0
30,HXgZYzItf1dg3U9osEk2wNbSlNuA6gEa.6451e1f34b19e...,Chrome 84.0,Curitiba,BR,2020-06-01 11:51:28,leads2b.com,2020-08-06 16:31:43,Windows,Parana,LA Criar Conta,6,66
31,KvlhZUYVslEa4GDBdOM0KcbnQg03v4QV.f2f55b40d6d78...,Chrome 83.0,Sao Jose,BR,2020-06-02 17:05:53,$direct,2020-06-03 09:21:16,Windows,Santa Catarina,LA Criar Conta,70,0


In [70]:
df_perfils.dtypes

distinct_id                         object
Browser                             object
city                                object
country_code                        object
created                     datetime64[ns]
initial_referring_domain            object
last_seen                   datetime64[ns]
os                                  object
region                              object
Source                              object
recencia                             int64
duracao                              int64
dtype: object

### Agora vamos para o Volume, ou seja, contar o número total de eventos feitos.

Mas antes, vamos dar um double-check nessa relação entre os dados de evento e de perfil.

In [72]:
df_eventos['distinct_id'].nunique()

10664

In [71]:
df_perfils['distinct_id'].nunique()

9251

Por que há mais usuários únicos registrados na tabela de perfils que na de eventos? Talvez tenha esquecido de remover os usuários internos da Leads na tabela de eventos.

In [8]:
df_eventos['distinct_id'].count()

934355

In [9]:
df_eventos.groupby('distinct_id').count()

,event,time
distinct_id,,
00JPlpZE7fwEBd02ckdN2USioJFEl48m.926fc147413241ccececdaf2737e3895,1,1
00lEp5f2lHSRkdmPtNUCNvqNwAoH9mV8.820a3738c36c6c1c0696db5d816c573b,83,83
012F3TlZie1i3OqXPiA2MMAMaHkecEgc.b2d86c6fd5ad2f852f1848e2099bcf37,6,6
01PAzfApVUdVGyxAzXO3paELJpAxnLWP.688c75d03aaa1d3a023df9c71fcf4462,1,1
01upcsg4zvEi61OOipUEIsNqEw6gY05N.85cba73c00485443335b4458481b3246,4,4
...,...,...
zvrE05epQyzByZ2a219ks4JyOeWsWswp.32bad93919c9a7c7f81332d9b0651e81,4,4
zxOPfy5Qdfe71ldmgqztzj5CxfPMHqWp.ce6613207f02022b565c117c00178fc0,26,26
zyAnzYL8sUBD3wNIMD6XvSd69JmCtCon.fb6c71cfa13c1e134cefc39a8850a49f,1,1


In [10]:
df_eventos['distinct_id'].value_counts().agg(['min', 'max', 'mean', 'median'])

min           1.000000
max       15284.000000
mean         87.617686
median       15.000000
Name: distinct_id, dtype: float64

Mas fora isso, o resto parece saudável. Vamos então criar o Volume contando na tabela de eventos quantas vezes um UniqueID aparece e jogando essa contagem de volta para a tabela de perfils.

In [13]:
count_eventos = df_eventos['distinct_id'].value_counts()

In [14]:
df_count_eventos = count_eventos.to_frame()

In [16]:
# gambiarras de Pandas muito feias, favor não ver
df_count_eventos.reset_index(level=0, inplace=True)
df_count_eventos.rename(columns={'distinct_id':'count_eventos'}, inplace=True)
df_count_eventos.rename(columns={'index':'distinct_id'}, inplace=True)

In [81]:
df_count_eventos.head()

,distinct_id,count_eventos
0,eO4IBKUGkkJXfLCvGFisADzik8cSlQqm.f26d9494eea07...,15284
1,NCSCcBljLTHR5zBkF84UtTFxsvOAhlvO.936cbcc681bb8...,9502
2,tIPIx8WFIYwrTzq3BUbM25inzz3CYqol.f17c1b26b6dff...,9239
3,QKK4JVKAB9BFV2yLVoPCccYAmXI9QwRs.88dcefc39941b...,8287
4,ictkhvWXdhEGz0ssGMxlWWpBfhEQX5em.2c1677680b038...,7617


Parece saudável a contagem, vamos criar uma nova tabela com os dados de perfil e a contagem dos eventos.

In [82]:
df_consolidado = df_count_eventos.merge(df_perfils, on='distinct_id')

In [83]:
df_consolidado.head()

,distinct_id,count_eventos,Browser,city,country_code,created,initial_referring_domain,last_seen,os,region,Source,recencia,duracao
0,QKK4JVKAB9BFV2yLVoPCccYAmXI9QwRs.88dcefc39941b...,8287,Chrome 84.0,São Paulo,BR,2020-05-08 07:11:46,$direct,2020-07-28 15:34:10,Windows,Sao Paulo,LA Criar Conta,15,81
1,1723852f6f284-05818830b468d6-134b3c29-49d40-17...,5793,Chrome 84.0,Sao Jose do Rio Preto,BR,2020-05-21 11:40:59,leads2b.com,2020-08-05 18:35:38,Windows,Sao Paulo,CC CNPJ - Painel de Dados - E-mails,7,76
2,1724d0fbffc130-045e6b955db998-d373666-100200-1...,4296,Chrome 83.0,Guarapuava,BR,2020-05-25 12:19:35,$direct,2020-06-05 09:17:50,Windows,Parana,LA Criar Conta,68,10
3,SGIRpUNfoHiJMGa2AO7VlWg6mjGlDGHt.27dd4e881e36a...,3668,Chrome 84.0,Londrina,BR,2020-05-11 08:15:58,$direct,2020-08-12 13:35:45,Windows,Parana,LA Criar Conta,0,93
4,17231edb3c2d1-023a308ef89129-d373666-1fa400-17...,3403,Chrome 83.0,Salto,BR,2020-05-20 05:52:36,$direct,2020-07-02 17:28:51,Windows,Sao Paulo,LA Criar Conta,41,43


### A Frequência é razoavelmente mais difícil de calcular, então vamos exportar o que temos até agora e continuar no próximo arquivo

In [85]:
df_consolidado.to_csv('data/03-perfils-com-rfdv.csv')